Natural Language Processing Assignment
Author: Raeed Asif
PoS Tagging with Hidden Markov Model

importing nltk and brown corpus

In [1]:
import nltk
from nltk.corpus import brown

download if! NTLK brown and google universal_tagset

In [2]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /Users/raeedasif/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/raeedasif/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

retrieving tagged_sentences from brown of category 'news' and with tagset of google

In [4]:
tagged_sentences = brown.tagged_sents(categories='news', tagset = 'universal')

Training and test set

In [5]:
train_data = tagged_sentences[:-500] # training set
test_data = tagged_sentences[-500:] # test set

extracting words and tags as a seperate list from test set

In [56]:
untagged_words = [word for sent in tagged_sentences for word, tag in sent]
test_base_tags = [tag for sent in tagged_sentences for word, tag in sent]

# Hidden Markov Model Trainer

training HMM model

In [8]:
from nltk.tag import hmm
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_data)

testing HMM model

In [82]:
temp=[]
temp2=[]
hmm_tagged_seq=[]
for word in untagged_words:
    temp_seq = tagger.tag(word.split())
    temp.append(temp_seq)
    
temp1 = [j for sub in temp for j in sub]
for j in temp1:
    temp2.append([i.replace('(', '') for i in j])
temp3 = [j for sub in temp2 for j in sub]
for i in range(1,len(temp3),2):
    hmm_tagged_seq.append(temp3[i])
print(hmm_tagged_seq)

['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', '.', 'DET', 'NOUN', '.', 'PRON', 'DET', 'NOUN', 'VERB', 'NOUN', '.', 'DET', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'PRON', 'DET', 'NOUN', 'NOUN', 'NOUN', '.', 'DET', 'VERB', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', '.', '.', 'VERB', 'DET', 'NOUN', 'CONJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PRT', 'VERB', 'NOUN', 'ADP', 'ADJ', '.', 'NOUN', '.', 'ADP', 'DET', 'ADJ', 'NOUN', 'DET', 'VERB', 'VERB', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '.', '.', 'ADV', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', '.', 'DET', 'NOUN', 'VERB', '.', '.', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', '.', 'DET', 'NOUN', 'ADP', 'NOUN', 'CONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN

Accuracy testing

In [85]:
sample_test_check_hmm = [i for i, j in zip(test_base_tags, hmm_tagged_seq) if i == j]
acc = ((len(v_tagged_seq_flatten)- len(sample_test_check))/len(v_tagged_seq_flatten)*100)
print("accuracy:",(acc))

accuracy: 84.91551994426058


# Vertibi algorithim 

adding a START and END tag in the data 

In [12]:
tagged_words = [ ]
all_tags = [ ]

for sent in train_data:
    tagged_words.append( ("START", "START") )
    all_tags.append("START")
    for (word, tag) in sent:
        all_tags.append(tag)
        tagged_words.append( (tag, word) )
    tagged_words.append( ("END", "END") )
    all_tags.append("END")

## trasition probabilities [ P(t_{i} | t_{i-1}) = C(t_{i-1}, t_{i})/C(t_{i-1}) ]: 

In [16]:
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(all_tags)) # C(t_{i-1}, t_{i}):
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist) # P(t_{i} | t_{i-1})

## emission probabilities [ P(w_{i} | t_{i}) =  C(t_{i}, w_{i}) / C(t_{i}) ] 

In [17]:
cfd_tagwords = nltk.ConditionalFreqDist(tagged_words) # C(t_{i}, w_{i})
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)  # P(w_{i} | t_{i})

Vertibi Algorithm:-

In [18]:
import numpy as np
def viterbi(sentence):
    # Step 1. initialization step
    distinct_tags = np.array(list(set(all_tags)))
    tagslen = len(distinct_tags)

    sentlen = len(sentence)
    viterbi = np.zeros((tagslen, sentlen+1) ,dtype=float)
    backpointer = np.zeros((tagslen, sentlen+1) ,dtype=np.uint32)



    # Step 1. initialization step
    for s, tag in enumerate(distinct_tags):
        viterbi[s,0] =  cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
        backpointer[s,0] = 0
        #print("Viterbi probability V( {1} ,{0} ) = {2}".format(sentence[0],tag,  viterbi[s,0]) )
    #print('============================')

    # Step 2. recursion step
    for t in range(1, sentlen):
        for s, tag in enumerate(distinct_tags):
            current_viterbi = np.zeros( tagslen ,dtype=float)
            for sprime, predtag in enumerate(distinct_tags):
                current_viterbi[sprime] = viterbi[sprime,t-1] * \
                                          cpd_tags[predtag].prob(tag) * \
                                          cpd_tagwords[tag].prob(sentence[t])
            backpointer[s,t] = np.argmax(current_viterbi)
            viterbi[s,t] = max(current_viterbi)
            #print("Viterbi probability V( {1} ,{0} ) = {2}".format(sentence[t],tag,  viterbi[s,t]))

        #print('============================')


    # Step 3. termination step
    current_viterbi = np.empty( tagslen ,dtype=float)
    ind_of_end = -1
    for s, tag in enumerate(distinct_tags):
        if tag == "END":
            ind_of_end  = s
        current_viterbi[s] = viterbi[s,sentlen-1] * cpd_tags[tag].prob("END")

    backpointer[ind_of_end,sentlen] = np.argmax(current_viterbi)
    viterbi[ind_of_end,sentlen] = max(current_viterbi)

    # Step 3. backtrace the path
    best_tagsequence = [ ]
    prob_tagsequence = viterbi[ind_of_end,sentlen]
    prevind  = ind_of_end
    for t in range(sentlen,0,-1):
        prevind = backpointer[prevind,t]
        best_tagsequence.append(distinct_tags[prevind])
    best_tagsequence.reverse()

    return best_tagsequence, prob_tagsequence

testing set on vertibi

In [32]:
nltk.download('punkt')
v_tagged_seq=[]

for word in untagged_words:
	sentence =  nltk.word_tokenize(word)
	best_tagsequence,prob_tagsequence = viterbi(sentence)
	v_tagged_seq.append(best_tagsequence)

v_tagged_seq_flatten = [j for sub in v_tagged_seq for j in sub]

print(v_tagged_seq_flatten)

[nltk_data] Downloading package punkt to /Users/raeedasif/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP', 'PRON', 'PRON', 'ADJ', 'NOUN', 'NOUN', 'VERB', '.', 'DET', 'NOUN', '.', 'DET', 'DET', 'NOUN', 'VERB', 'NOUN', '.', 'DET', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'DET', 'DET', 'NOUN', 'NOUN', 'NOUN', '.', 'DET', 'VERB', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', '.', '.', 'VERB', 'DET', 'NOUN', 'PRON', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PRT', 'VERB', 'NOUN', 'ADP', 'ADJ', '.', 'NOUN', '.', 'ADP', 'DET', 'ADJ', 'NOUN', 'DET', 'VERB', 'VERB', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'PRON', 'PRON', '.', '.', 'ADV', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', '.', 'DET', 'NOUN', 'VERB', '.', '.', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', '.', 'DET', 'NOUN', 'ADP', 'NOUN', 'PRON', 'DET', 'NOUN', 'ADP

Accuracy testing

In [86]:
sample_test_check = [i for i, j in zip(test_base_tags, v_tagged_seq_flatten) if i == j]
acc = ((len(v_tagged_seq_flatten)- len(sample_test_check))/len(v_tagged_seq_flatten)*100)
print("accuracy:",(acc))

accuracy: 84.91551994426058
